In [1]:
# !pip install bs4
# !pip install requests
# !pip install tqdm
# !pip install pandas
# !pip install datetime 
# !pip install json 
# !pip install pymysql
# !pip install sqlalchemy

In [2]:
import winsound
# import pygame
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication
import zlib

from webscraper.scrap_modules import scraper_ds as mybib


%load_ext autoreload
%autoreload 2

In [3]:
# pygame.mixer.init()
# pygame.mixer.music.load("path/to/your/audiofile.mp3")
# pygame.mixer.music.play()
# while pygame.mixer.music.get_busy():
#         pygame.time.Clock().tick(10)  # Wait for the music to finish

In [4]:
# try:
#     # Your code here
#     x = 1 / 0  # This will cause a ZeroDivisionError
# except Exception as e:
#     print(f"An error occurred: {e}")
#     winsound.PlaySound("alarm2.wav", winsound.SND_FILENAME)

### Import keywords

In [16]:
keyword_list = mybib.import_keyword_list()
keyword_list

['Data%20Governance%20Specialist',
 'Statistical%20Analyst',
 'Research%20Scientist',
 'Data%20Journalism',
 'Data%20Product%20Manager',
 'AI%20Engineer%20(Artificial%20Intelligence)',
 'Data%20Strategist',
 'Data%20Warehouse%20Engineer',
 'Predictive%20Analyst',
 'Operations%20Analyst']

### Scrap Prep
- connect with MySQL to get id's
- create empty DataFrame for saving scraping results
- print results for check

In [6]:
try:
    with open("webscraper/webscrap_data/soup_dict.pkl", "rb") as file:
        soup_dict = pickle.load(file)
    with open("webscraper/temp_data/soup_dict_backup.pkl", "wb") as file:
        pickle.dump(soup_dict, file)
except:
    print("No soup dictionary found. Created empty dictionary")         

# try:
#     with open("webscraper/webscrap_data/database.pkl", "rb") as file:
#         database = pickle.load(file)
#     with open("webscraper/temp_data/database_backup.pkl", "wb") as file:
#         pickle.dump(database, file)
        
#     id_control = database['id'].tolist()
# except:
#     print("No dataframe found.")
#     columns = ['id','title', 'company', 'city', 'posting_date', 'job_description', 'seniority_level', 'job_function', 'industries', 'scraping_date', 'url', 'keyword']
#     database = pd.DataFrame(columns=columns)
#     id_control = []   
        
try:
    with open("webscraper/webscrap_data/keyword_dict.pkl", "rb") as file:
        keyword_dict = pickle.load(file)
    with open("webscraper/temp_data/keyword_dict_backup.pkl", "wb") as file:
        pickle.dump(soup_dict, file) 
except:
    print("No keyword dictionary found. Created empty dictionary")   

    
# scraper_df = mybib.initialize_empty_df()

In [7]:
print(soup_dict)
print(len(soup_dict))
print(keyword_dict)
print(len(keyword_dict))

{'3719620327': <Response [200]>, '3720365801': <Response [200]>, '3672508925': <Response [200]>, '3695466781': <Response [200]>, '3585738180': <Response [200]>, '3585734564': <Response [200]>, '3704916810': <Response [200]>, '3585734595': <Response [200]>, '3698396795': <Response [200]>, '3687813018': <Response [200]>, '3714114129': <Response [200]>, '3708672280': <Response [200]>, '3707596225': <Response [200]>, '3706371923': <Response [200]>, '3694298306': <Response [200]>, '3641771902': <Response [200]>, '3701510210': <Response [200]>, '3582381366': <Response [200]>, '3674803883': <Response [200]>, '3725401052': <Response [200]>, '3713235511': <Response [200]>, '3720128757': <Response [200]>, '3694036752': <Response [200]>, '3712833846': <Response [200]>, '3725985159': <Response [200]>, '3713068792': <Response [200]>, '3690755622': <Response [200]>, '3710653394': <Response [200]>, '3688655136': <Response [200]>, '3697730282': <Response [200]>, '3694113016': <Response [200]>, '367931

In [8]:
print(soup_dict)
id_control = []
for item in soup_dict.keys():
    id_control.append(item)

{'3719620327': <Response [200]>, '3720365801': <Response [200]>, '3672508925': <Response [200]>, '3695466781': <Response [200]>, '3585738180': <Response [200]>, '3585734564': <Response [200]>, '3704916810': <Response [200]>, '3585734595': <Response [200]>, '3698396795': <Response [200]>, '3687813018': <Response [200]>, '3714114129': <Response [200]>, '3708672280': <Response [200]>, '3707596225': <Response [200]>, '3706371923': <Response [200]>, '3694298306': <Response [200]>, '3641771902': <Response [200]>, '3701510210': <Response [200]>, '3582381366': <Response [200]>, '3674803883': <Response [200]>, '3725401052': <Response [200]>, '3713235511': <Response [200]>, '3720128757': <Response [200]>, '3694036752': <Response [200]>, '3712833846': <Response [200]>, '3725985159': <Response [200]>, '3713068792': <Response [200]>, '3690755622': <Response [200]>, '3710653394': <Response [200]>, '3688655136': <Response [200]>, '3697730282': <Response [200]>, '3694113016': <Response [200]>, '367931

In [9]:
# print("Overview SQL Database")
# display(database.head(10))
# print(database.shape)

# print("Empty Dataframe")
# display(scraper_df.head())

# new_dataframe.shape

In [10]:
# id_control = database['id'].tolist()
id_control

['3719620327',
 '3720365801',
 '3672508925',
 '3695466781',
 '3585738180',
 '3585734564',
 '3704916810',
 '3585734595',
 '3698396795',
 '3687813018',
 '3714114129',
 '3708672280',
 '3707596225',
 '3706371923',
 '3694298306',
 '3641771902',
 '3701510210',
 '3582381366',
 '3674803883',
 '3725401052',
 '3713235511',
 '3720128757',
 '3694036752',
 '3712833846',
 '3725985159',
 '3713068792',
 '3690755622',
 '3710653394',
 '3688655136',
 '3697730282',
 '3694113016',
 '3679310304',
 '3704998318',
 '3711964211',
 '3704824751',
 '3722320318',
 '3719931304',
 '3699420235',
 '3582374933',
 '3674539739',
 '3715330729',
 '3586236970',
 '3463183538',
 '3698393103',
 '3638551593',
 '3712061807',
 '3720129659',
 '3590312287',
 '3624508875',
 '3709617062',
 '3709613515',
 '3705398931',
 '3723216390',
 '3654246723',
 '3698385819',
 '3718212892',
 '3725625612',
 '3726120710',
 '3714476818',
 '3710721213',
 '3714499999',
 '3706373793',
 '3724502483',
 '3652093754',
 '3720171940',
 '3709609985',
 '36610156

In [11]:
# sql_data = mybib.connect_sql_database()
# scraper_df = mybib.initialize_empty_df()

# print("Overview SQL Database")
# display(sql_data.head(5))
# display(sql_data.tail(5))

# print("Empty Dataframe")
# display(scraper_df)

# print("Overview ID control")
# id_control = sql_data['id'].tolist()
# id_control



    

In [12]:
# new_scraper_df, new_soup_dict, new_keyword_dict = mybib.total_loop_rough_keywords(keyword_list, scraper_df, id_control, soup_dict, keyword_dict)

In [13]:
# all time https://www.linkedin.com/jobs/search?keywords={key}&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=104944500&f_TPR=&distance=25&f_E=2%2C3%2C4&f_JT=F&position=1&pageNum=0
# past month https://www.linkedin.com/jobs/search?keywords=Data%20Analayst&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=104944500&f_TPR=r604800&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0

In [14]:
# id_list


In [17]:
# def total_loop_rough_keywords(keyword_list, scraper_df, id_control, soup_dict, keyword_dict): 

# links = {}
try:
    for key in keyword_list:
        first_url = f"https://www.linkedin.com/jobs/search?keywords={key}&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=104944500&f_TPR=&distance=25&f_E=2%2C3%2C4&f_JT=F&position=1&pageNum=0"
        key_name = key.replace('%20', " ")
            
        response = requests.get(first_url) # first request for keyword
        response.status_code # 200 status code means OK!
        soup = BeautifulSoup(response.content, "html.parser")
        
        number_of_results = soup.find('span', class_="results-context-header__job-count").text # check number of searching results
        numb = int(number_of_results.replace(",", "").replace("+", ""))

        backend_call_url_list = []
        backend_call_url_list = mybib.create_backend_links(first_url, numb, key_name) # create list with sublinks to select different pages 
        
        
        # with open(f'{key_name}_backend_urls.pkl', 'wb') as file:
        #     pickle.dump(backend_call_url_list, file)
        
        id_list = mybib.get_id_list(backend_call_url_list) # get job id's from all pages
        

        with tqdm(total=len(id_list), desc="Starting") as pbar:
        
            for id in id_list:
                counter = 0
                backupsave = [25, 50, 75, 100, 125, 150, 175, 200, 300] 
                if counter in backupsave:
                    new_dataframe = mybib.export_data(soup_dict, keyword_dict)
                    scraper_df = pd.DataFrame(columns=scraper_df.columns)
                    print("Updated dataframe")
                dynamic_text = f"id {id}" # text for tqdm progress bar status
                pbar.set_description(dynamic_text) # change text
                pbar.update(1)
                
                if str(id) not in id_control:
                    try:
                        # new_row_dict, id_control, response = mybib.get_all_job_information(id, id_control) # 
                        response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}')
                        wait_time = randint(1,1500)
                        sleep(wait_time/1000)
                        try: 
                            soup_dict[id] = response
                            # If the key 'id' doesn't exist, initialize it with a list containing the new key
                        except Exception as e:
                            # Handle any exceptions and provide informative error messages
                            raise ValueError(f"Error occurred: {e}")  
                        
                        try: 
                            if id in keyword_dict:
                                keyword_dict[id].append(key_name)
                            # If the key 'id' doesn't exist, initialize it with a list containing the new key
                            else:
                                keyword_dict[id] = [key_name]

                        except Exception as e:
                            # Handle any exceptions and provide informative error messages
                            raise ValueError(f"Error occurred: {e}")                         
                        # display(soup_dict)
                        
                        # try:
                        #     new_row_df = pd.DataFrame([new_row_dict])
                        #     scraper_df = pd.concat([scraper_df, new_row_df], ignore_index=True)
                        #     # display(dataframe)
                        # except Exception as e:
                        #     # Handle any exceptions and provide informative error messages
                        #     raise ValueError(f"Error occurred: {e}")  
                        
                        # try: 
                        #     id_control = id_control.append(id)
                        # except Exception as e:
                        #     # Handle any exceptions and provide informative error messages
                        #     raise ValueError(f"Error occurred: {e}")  

                    except Exception as e:
                    
                        
                        # with open(f'webscraper/temp_data/back_up_df.pkl', 'wb') as file:
                        #     json.dump(scraper_df, file=file)
                        # with open('webscraper/temp_data/soup_dict_backup.pkl', 'w') as f:
                        #     json.dump(soup_dict, f)
                        # with open('webscraper/temp_data/keyword_dict.pkl', 'w') as f:
                        #     json.dump(keyword_dict, f)
                        # # with open('webscraper/temp_data/id_list_backup.pkl', "wb") as file:
                        #     pickle.dump(id_list, file=file)
                        raise ValueError(f"Error occurred: {e}")
                else:
                    pbar.set_description(f"Will skip {id} because is already in the dataset.")
                counter += 1    
        new_dataframe = mybib.export_data(soup_dict, keyword_dict)
        print("\n \n")
except Exception as e:
    print(f"An error occurred: {e}")
    winsound.PlaySound("alarm2.wav", winsound.SND_FILENAME)

Start creating backend links for Data Governance Specialist. Total of found pages: 80



  4%|███                                                                                | 3/80 [00:05<02:28,  1.92s/it]

  8%|██████▏                                                                            | 6/80 [00:12<02:28,  2.01s/it]

 21%|█████████████████▍                                                                | 17/80 [00:39<02:24,  2.30s/it]

 41%|█████████████████████████████████▊                                                | 33/80 [01:19<01:36,  2.06s/it]

...


 

Start creating backend links for Operations Analyst. Total of found pages: 15



 20%|████████████████▌                                                                  | 3/15 [00:08<00:38,  3.25s/it]

 27%|██████████████████████▏                                                            | 4/15 [00:11<00:31,  2.87s/it]

 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [00:33<00:01,  1.95s/it]

id 3659009967: 100%|█████████████████████████████████████████████████████████████████| 366/366 [03:09<00:00,  1.93it/s]


In [ ]:
# display(new_dataframe.shape)
display(len(soup_dict))
display(len(keyword_dict))

In [ ]:
new_dataframe['id'].value_counts()

In [ ]:
new_dataframe['id'].value_counts()

new_dataframe[new_dataframe['id']=="3670882403"]

In [ ]:
df_unique = new_dataframe.drop_duplicates(subset=['id'])
df_unique


In [ ]:

with open("webscraper/webscrap_data/database.pkl" , "wb") as f:
    pickle.dump(df_unique, f)

In [ ]:
columns = new_dataframe.columns

for col in columns:
    null = new_dataframe[col].isna().sum()
    print(f"{round(null / len(new_dataframe[col]) * 100,1)}%    {col}:")

In [ ]:
keywords_input = ['Data%20Analyst']
keywords = []


# with open(f"webscrap_data/{filename}", 'rb') as file:
#        scraper_df = pickle.load(file)

In [ ]:
# https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0

    # first_url = f"https://www.linkedin.com/jobs/search?keywords={key}&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&distance=25&f_JT=F&f_E=2%2C3&f_PP=106967730&position=1&pageNum=0"

In [ ]:
first_url = f"https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0"



In [ ]:

# soup_dict = {}
# links = {}
# for key in keywords:
#     # first_url = f"https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0"
#     key_name = key.replace('%20', " ")
        
#     # response = requests.get(first_url)
#     # response.status_code # 200 status code means OK!
#     # soup = BeautifulSoup(response.content, "html.parser")
    
#     # number_of_results = soup.find('span', class_="results-context-header__job-count").text
#     # numb = int(number_of_results.replace(",", "").replace("+", ""))
#     # # print(numb) # number of job post found 
    
#     # backend_call_url_list = create_backend_links(first_url, numb, key_name)
#     # with open(f'{key_name}_backend_urls.pkl', 'wb') as file:
#     #     pickle.dump(backend_call_url_list, file)
    
#     # id_list = get_id_dict(backend_call_url_list)
    
#     with tqdm(total=len(id_list), desc="Starting") as pbar:
#         for id in id_list:
#             dynamic_text = f"Progressing id: {id}"
#             pbar.set_description(dynamic_text)
#             if id not in id_control:
#                 try:
#                     scraper_df, id_control, small_soup = get_all_job_information(key_name, scraper_df, id, id_control)
#                     wait_time = randint(1,3000)
#                     pbar.set_description(f"Sleep {wait_time} seconds")
#                     sleep(wait_time/1000)
                    
#                     soup_dict[id] = small_soup
#                 except:
#                     print(f"Error when scraping data from id: {id}, temp backups are created in temp_data.")
#                     scraper_df_json = scraper_df.to_json('dataframe.json', orient='split', date_format='iso', indent=4)
#                     with open(f'temp_data/back_up_df.json', 'wb') as file:
#                         json.dump(scraper_df_json, file=file)
#                     with open('temp_data/small_soup_backup.json', 'w') as f:
#                         json.dump(small_soup, f)
#                     with open('temp_data/id_list_backup.pkl', "wb") as file:
#                         pickle.dump(links, file=file)
#             else:
#                 pbar.set_description(f"Will skip {id} because is already in the dataset.")
#             pbar.update(1)
    
#     current_date = datetime.now()
#     time = current_date.strftime("%Y-%m-%d_%H-%M-%S")
#     time

#     with open(f'webscrap_data/webscrap{str(time)}.pkl', 'wb') as file:
#         pickle.dump(scraper_df, file=file)
#     display(scraper_df.tail(5))
#     wait_time = randint(1,10000)
#     print("I will sleep for " + str(wait_time/1000) + " seconds.")
#     sleep(wait_time/1000)


In [ ]:
# use zlib.compress put into dict with id 
# pickle dict 

response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3717202675')


In [ ]:
soup_dict['3719620327']

In [ ]:
response = soup_dict['3719620327']

soup = BeautifulSoup(response.content, "html.parser")
soup

In [ ]:
soup.find("section",{"class":"show-more-less-html"}).text.strip()